#### Installations of libraries and Setting environment variables

In [2]:
from google.colab import drive
import torch
import json
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# faiss is for dot product
!pip install sentence-transformers faiss-cpu rouge_score nltk -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.8 MB/s eta 0:00:00


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cpu


In [ ]:
!hf auth login

#### Variables

In [5]:
file_path = '/content/drive/MyDrive/Uni/DS 5983/Final Project/Data' # nicholai
# file_path = '/content/drive/MyDrive/Final Project/Data' # shreya
# file_path = '/content/drive/MyDrive/DS5983-FinalProjcet' # tyree

In [6]:
vector_file_name = 'VectorDB.json'
# vector_file_name = 'VectorDB_Mar_200_Fixed.json' # Marsillia embedding, fixed chunk size of (up to) 200 words, no overlap between chunks
# vector_file_name = 'VectorDB_Mar_100_Window.json' # Marsillia embedding, fixed chunk size of (up to) 100 words, with 10 words of overlap always
# vector_file_name = 'VectorDB_finbert.json'

In [7]:
EMBEDDING_MODEL = "sujet-ai/Marsilia-Embeddings-EN-Large" # this was the first model
# EMBEDDING_MODEL = "ProsusAI/finbert" # trained on sentiment classification, so it needed to append mean-pooling layer for sentence transformer
# EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # General purpose, fast
# EMBEDDING_MODEL = "sentence-transformers/all-mpnet-base-v2"  # Higher quality

#### Queries and target

In [8]:
# 30 queries with target chunks.
queries_and_target = {
    "From the January 2025 meeting, what is the Fed's current stance on the federal funds rate, and when might they cut?": [
        "All participants viewed it as appropriate to maintain the target range for the federal funds rate at 4-1/4 to 4-1/2 percent.",
        "Participants indicated that, provided the economy remained near maximum employment, they would want to see further progress on inflation before making additional adjustments to the target range for the federal funds rate.",
    ],

    "From the January 2025 meeting, how does the Fed assess labor market conditions, such as unemployment and payroll gains?": [
        "Payroll gains had averaged 170,000 per month over the last three months of 2024 and the unemployment rate had stabilized at a relatively low level.",
        "Participants judged that labor market conditions had remained solid and that those conditions were broadly consistent with the Committee's goal of maximum employment",
        "The unemployment rate edged down to 4.1 percent in December,",
    ],

    "From the January 2025 meeting, what is the Federal Reserve’s plan for its balance sheet in terms of its security holdings?": [
        "Participants judged that it was appropriate to continue the process of reducing the Federal Reserve's securities holdings.",
        "Roll over at auction the amount of principal payments from the Federal Reserve's holdings of Treasury securities maturing in each calendar month that exceeds a cap of $25 billion per month.",
        "Reinvest the amount of principal payments from the Federal Reserve's holdings of agency debt and agency mortgage‑backed securities (MBS) received in each calendar month that exceeds a cap of $35 billion per month into Treasury securities.",
    ],

    "From the March 2025 meeting, how are tariffs and trade policy affecting the Federal Reserve’s outlook?": [
        "participants judged that inflation was likely to be boosted this year by the effects of higher tariffs, although significant uncertainty surrounded the magnitude and persistence of such effects.",
        "Several participants noted that the announced or planned tariff increases were larger and broader than many of their business contacts had expected.",
        "Several participants also noted that their contacts were already reporting increases in costs, possibly in anticipation of rising tariffs, or that their contacts had indicated willingness to pass on to consumers higher input costs that would arise from potential tariff increases.",
    ],

    "From the March 2025 meeting, why did the Federal reserve slow the pace of balance sheet runoff and what does this signal?": [
        "Almost all participants supported a slowing of the pace of runoff at this meeting",
        "Roll over at auction the amount of principal payments from the Federal Reserve's holdings of Treasury securities maturing in March that exceeds a cap of $25 billion per month. Beginning on April 1, roll over at auction the amount of principal payments from the Federal Reserve's holdings of Treasury securities maturing in each calendar month that exceeds a cap of $5 billion per month.",
        "Some participants noted that a slower pace of runoff would also help guard against reserve scarcity emerging with little advance notice during a period of potentially rapid increase in the TGA.",
        "Most participants noted the importance of clearly communicating that slowing the runoff pace had no implications for the stance of monetary policy",
    ],

    "From the March 2025 meeting, what are the Federal Reserve's main concerns about economic risks right now?": [
        "participants generally saw increased downside risks to employment and economic growth and upside risks to inflation while indicating that high uncertainty surrounded their economic outlooks.",
        "The staff judged that the risks around the baseline projections for economic activity and employment had tilted to the downside.",
        "Almost all participants pointed out that many market- or survey-based measures of near-term expected inflation had increased recently.",
        "Several participants emphasized that elevated inflation could prove to be more persistent than expected.",
        "Some participants observed, however, that the Committee may face difficult tradeoffs if inflation proved to be more persistent while the outlook for growth and employment weakened",
    ],

    "From the May 2025 FOMC meeting, how severe were the tariff announcements and what was the market impact?": [
        "Participants assessed that the tariff increases announced so far had been significantly larger and broader than they had anticipated.",
        "The manager observed that market participants appeared to interpret recently announced trade policy changes as a negative supply shock that could restrain domestic activity relative to foreign activity in the near term.",
        "respondents to the Open Market Desk's Survey of Market Expectations had materially lowered their gross domestic product (GDP) forecasts and raised their inflation forecasts for this year while significantly increasing the probability they placed on a recession occurring within the next six months.",
        "Liquidity in foreign exchange markets deteriorated but was roughly in line with the historical relationship between liquidity and measures of market volatility.",
    ],

    "From the May 2025 FOMC meeting, why did the dollar weaken despite higher U.S. rates, and what does this mean?": [
        "The dollar depreciated substantially against most major currencies, as the trade-weighted broad dollar index declined over 2 percent.",
        "The manager noted that the dollar had depreciated even though U.S. interest rates had risen more than foreign interest rates and prices of risky assets had declined, which historically have been associated with dollar appreciation.",
        "Market contacts attributed the decline to increased foreign exchange hedging prompted by heightened policy uncertainty and concerns that trade policy could pose greater downside risks to the U.S. than to other economies.",
        "Some participants commented on a change from the typical pattern of correlations across asset prices during the first half of April, with longer-term Treasury yields rising and the dollar depreciating despite the decline in the prices of equities and other risky assets. These participants noted that a durable shift in such correlations or a diminution of the perceived safe-haven status of U.S. assets could have long-lasting implications for the economy.",
    ],

    "From the May 2025 meeting, how high is recession risk according to the Federal Reserve?": [
        "The staff viewed the possibility that the economy would enter a recession to be almost as likely as the baseline forecast",
        "The labor market was expected to weaken substantially, with the unemployment rate forecast moving above the staff's estimate of its natural rate by the end of this year and remaining above the natural rate through 2027",
        "Participants judged that downside risks to employment and economic activity and upside risks to inflation had risen, primarily reflecting the potential effects of tariff increases",
        "The staff continued to note the large amount of uncertainty surrounding trade policy and other economic policies and now viewed the uncertainty around the projection as elevated relative to the average over the past 20 years.",
    ],

    "From the June 2025 FOMC meeting, has the Federal Reserve more seriously considered rate cuts and are they coming soon?": [
        "Most participants assessed that some reduction in the target range for the federal funds rate this year would likely be appropriate",
        "A couple of participants noted that, if the data evolve in line with their expectations, they would be open to considering a reduction in the target range for the policy rate as soon as at the next meeting",
        "The median respondent's modal path for the federal funds rate in the June survey shifted higher through 2026 and implied two 25 basis point rate cuts both this year and next year",
    ],

    "From the June 2025 FOMC meeting, what happened with the trade tensions and tariffs since May?": [
        "Markets were attentive to the de-escalation of trade tensions",
        "In mid-May, the U.S. and China agreed to a 90-day reduction in bilateral tariffs, and recent indicators suggested that this change led to a rebound in trade flows",
        "Participants observed that uncertainty about the economic outlook had diminished amid a reduction in announced and expected tariffs, which appeared to peak in April and had subsequently declined",
        "The median respondent's expectations for real gross domestic product (GDP) growth and personal consumption expenditures (PCE) inflation for 2025 retraced some of the moves that occurred after the April tariff announcements",
    ],

    "From the June 2025 FOMC meeting, how much has inflation improved and what's the outlook?": [
        "Total consumer price inflation—as measured by the 12-month change in the PCE price index—was estimated to have been 2.3 percent in May, based on the consumer and producer price indexes",
        "Core PCE price inflation, which excludes changes in consumer energy prices and many consumer food prices, was 2.6 percent in May",
        "Both total and core inflation were lower than at the beginning of the year",
        "Some participants observed that services price inflation had moved down recently, while goods price inflation had risen",
        "Participants noted that longer-term inflation expectations continued to be well anchored",
    ],


    "What are Amazon’s total capitalized costs for video and music for the years 2023 and 2024?": [
        "The total capitalized costs of video, which is primarily released content, and music as of December 31, 2023 and 2024 were $ 17.4 billion and $ 19.6 billion.",
    ],

    "How many shareholders did Apple report having in 2024?": [
        "As of October 18, 2024, there were 23,301 shareholders of record.",
        "Prompt: How did the net sales of Apple’s products and services change from 2023 to 2024?",

    ],

    "How did the net sales of Apple’s products and services change from 2023 to 2024?": [
        "iPhone net sales were relatively flat during 2024 compared to 2023.",
        "Mac net sales increased during 2024 compared to 2023 due primarily to higher net sales of laptops.",
        "iPad net sales decreased during 2024 compared to 2023 due primarily to lower net sales of iPad Pro and the entry-level iPad models, partially offset by higher net sales of iPad Air.",
        "Wearables, Home and Accessories net sales decreased during 2024 compared to 2023 due primarily to lower net sales of Wearables and Accessories.",
        "Services net sales increased during 2024 compared to 2023 due primarily to higher net sales from advertising, the App Store ® and cloud services.",
    ],

    "Describe the change in net sales for the iPhone, Mac, and iPad from 2023 to 2024.": [
        "iPhone net sales were relatively flat during 2024 compared to 2023.",
        "Mac net sales increased during 2024 compared to 2023 due primarily to higher net sales of laptops.",
        "iPad net sales decreased during 2024 compared to 2023 due primarily to lower net sales of iPad Pro and the entry-level iPad models, partially offset by higher net sales of iPad Air.",
        "Prompt: What were the amounts of dividends paid to stockholders of Alphabet Inc.’s Class A, Class B, and Class C shares for the year of 2024?",
        "Dividend payments to stockholders of Class A, Class B, and Class C shares were $3.5 billion, $519 million, and $3.3 billion , respectively, for the year ended December 31, 2024.",
        "Prompt: How did Alphabet Inc.’s revenue change from 2023 to 2024? What was the change driven by?",
        "Revenues were $350.0 billion, an increase of 14% year over year, primarily driven by an increase in Google Services revenues of $32.4 billion, or 12%, and an increase in Google Cloud revenues of $10.1 billion, or 31%.",
    ],

    "How did Meta’s total revenue change from 2023 to 2024? What was it driven by?": [
        "Total revenue for 2024",
        "was $164.50 billion, an increase of 22% compared to 2023, due to an increase in advertising revenue.",
    ],

    "How much in dividends did Meta pay in 2024?": [
        "Beginning in February 2024, our board of directors declared a quarterly cash dividend of $0.50 per share to the holders of our Class A and Class B common stock.",
        "RSUs granted on or after March 1, 2024 under our 2012 Equity Incentive Plan (Amended 2012 Plan), which was most recently amended in May 2024, are entitled to dividend equivalent rights.",
        "During the year ended December 31, 2024, total dividend and dividend equivalent payments were $4.38 billion and $691 million for Class A and Class B common stock, respectively.",
    ],

    "What are the highlights of Microsoft’s performance from fiscal year 2025 compared with fiscal year 2024?": [
        "Fiscal Year 2025 Compared with Fiscal Year 2024",
        "Revenue increased $36.6 billion or 15% with growth across each of our segments.",
    ],

    "How much in dividends did Microsoft pay to shareholders in 2025 and 2024?": [
        "During fiscal years 2025 and 2024, our Board of Directors declared dividends totaling $24.7 billion",
        "and $22.3 billion, respectively.",
        "Prompt: How much has Microsoft spent on advertising in the past few years?",
        "Advertising expense was $ 2.1 billion, $ 1.7 billion, and $ 904 million in fiscal years 2025, 2024, and 2023, respectively.",
    ],

    "How did the Fed's stance on interest rate cuts evolve throughout 2025?": [
        "Participants indicated that, provided the economy remained near maximum employment, they would want to see further progress on inflation before making additional adjustments to the target range for the federal funds rate",
        "Participants noted that policy decisions were not on a preset course and were conditional on the evolution of the economy",
        "With economic growth and the labor market still solid and current monetary policy restrictive, participants assessed that the Committee was well positioned to wait for more clarity on the outlook for inflation and economic activity",
        "In discussing the outlook for monetary policy, participants remarked that uncertainty about the net effect of an array of government policies on the economic outlook was high, making it appropriate to take a cautious approach",
        "In considering the outlook for monetary policy, participants agreed that with economic growth and the labor market still solid and current monetary policy moderately restrictive, the Committee was well positioned to wait for more clarity on the outlooks for inflation and economic activity",
        "Participants agreed that uncertainty about the economic outlook had increased further, making it appropriate to take a cautious approach",
        "Most participants assessed that some reduction in the target range for the federal funds rate this year would likely be appropriate",
        "A couple of participants noted that, if the data evolve in line with their expectations, they would be open to considering a reduction in the target range for the policy rate as soon as at the next meeting",
    ],

    "What was Google's total revenue in Q2 2025?": [
        "Consolidated revenue of $96.4 billion increased by 14% or 13% in constant currency.",
        "Consolidated Revenue: $96.4 billion, up 14%, reflecting double-digit gains in Search, YouTube, subscriptions, and Cloud.",
    ],

    "How many GitHub Copilot users does Microsoft have?": [
        "All-up, we now have over 15 million GitHub Copilot users, up over 4X year-over-year.",
        "And both digital natives like Twilio and enterprises like Cisco, HPE, SkyScanner, and Target, continue to choose GitHub Copilot to equip their developers with AI throughout the entire dev lifecycle.",
    ],

    "What is Amazon's current AWS revenue run rate?": [
        "AWS now has an annualized revenue run rate of more than $117 billion.",
        "Moving next to our AWS segment, revenue was $29.3 billion in Q1, an increase of 17% year-over-year. AWS now has an annualized revenue run rate of more than $117 billion.",
        "AWS grew 17% year-over-year in Q1 and now sits at a $117 billion annualized revenue run rate.",
    ],

    "Compare the AI infrastructure investments across Google, Meta, and Amazon in 2025": [
        "Google: Moving to investments, given the strong demand for our cloud products and services, we now expect to invest approximately $85 billion in CapEx in 2025, up from a previous estimate of $75 billion.",
        "Amazon: Now, turning to our cash CapEx, which was $24.3 billion in Q1. The majority of the spend is to support the growing need for technology infrastructure. This primarily relates to AWS as we invest to support demand for our AI services, and increasingly in custom silicon like Trainium.",
        "Meta: We anticipate our full year 2025 capital expenditures, including principal payments on finance leases, will be in the range of $64-72 billion, increased from our prior outlook of $60-65 billion.",
    ],

    "What are the main growth drivers for cloud services mentioned by AWS and Google Cloud?": [
        "Google Cloud: Google Cloud backlog increased 18% sequentially in Q2 and 38% year over year, reaching $106 billion at the end of the quarter. This growth was driven by strong demand for our products and services from both new and existing customers.",
        "Google Cloud: One, the number of deals over $250 million doubling year over year. Two, in the first half of 2025, we signed the same number of deals over $1 million that we did in all of 2024. Three, the number of new GCP customers increased by nearly 28% quarter over quarter.",
        "AWS: During the first quarter, we continue to see growth in both Generative AI business and non-Generative AI offerings. As companies turn their attention to newer initiatives, bring more workloads to the cloud, restart or accelerate existing migrations from on-premises to the cloud, and tap into the power of Generative AI.",
    ],

    "How are companies addressing AI capacity constraints and what solutions are they implementing?": [
        "Google: We have been working hard to increase capacity and have improved the pace of server deployment. We expect to remain in a tight demand-supply environment going into 2026.",
        "Amazon: At the bottom layer for those building models, our new custom AI chip Trainium 2 is starting to lay in capacity in larger quantities with significant appeal and demand... So, the 30% to 40% better price performance that Trainium 2 offers versus other GPU based instances is compelling.",
        "Microsoft: In our AI services, while we continue to bring datacenter capacity online as planned, demand is growing a bit faster. Therefore, we now expect to have some AI capacity constraints beyond June.",
        "Meta: So even with the capacity that we're bringing online in 2025, we are having a hard time meeting the demand that teams have for compute resources across the company.",
    ],

    "What are the different approaches to AI model efficiency and cost reduction across these tech companies?": [
        "Amazon: For AI to be as successful as we believe it can be, the price of inference needs to come down significantly. We consider this part of our mission and responsibility to help make it so.",
        "Google: As we ramp our AI investments, we continue to focus on driving improvements in productivity and efficiency to offset growth in technical infrastructure-related expenses, particularly from higher depreciation.",
        "Meta: A big focus of this work will be on developing increasingly efficient recommendation systems so that we can continue scaling up the complexity and compute used to train our models while avoiding diminishing returns.",
        "Microsoft: We continue to optimize and drive efficiencies across every layer – from DC design, to hardware and silicon, to systems software, to model optimization – all towards lowering costs and increasing performance.",
    ],

    "How are macroeconomic uncertainties affecting capital expenditure decisions in Q2 2025?": [
        "Meta: Before moving to our financial guidance, I want to acknowledge the dynamic macro environment and note that our range reflects the potential for a wider set of outcomes.",
        "Google: However, volatility in exchange rates could affect the impact of FX on Q3 revenue.",
        "Amazon: The external environment remains complex, and as we have done throughout our history, we are focused on the inputs that we can control to protect the customer experience.",
        "Meta: The higher costs we expect to incur for infrastructure hardware this year really comes from suppliers who source from countries around the world. And there's just a lot of uncertainty around this, given the ongoing trade discussions.",
    ],

    "Compare the monetization strategies for AI services across Google, Microsoft, Meta, and Amazon": [
        "Google: Advertisers that activate AI Max in search campaigns typically see 14% more conversions, and Campaigns using Smart Bidding Exploration see a 19% increase in conversions on average.",
        "Google: Management introduced the AgentSpace enterprise agent platform and reported over one million subscriptions booked ahead of general availability.",
        "Meta: We are also starting to introduce ads on unmonetized surfaces, like Threads, which we opened up to all eligible advertisers this month to reach people in over 30 different markets to start, including the US.",
        "Meta: In Q1, we introduced our new Generative Ads Recommendation model, or GEM, for ads ranking. This model uses a new architecture we developed that is twice as efficient at improving ad performance.",
    ],

    "What are the exact metrics Meta provided about Threads usage and monetization plans?": [
        "We are also starting to introduce ads on unmonetized surfaces, like Threads, which we opened up to all eligible advertisers this month to reach people in over 30 different markets to start, including the US.",
        "We don't expect Threads to be a meaningful driver of overall impression or revenue growth in 2025.",
        "We began testing using Llama in Threads recommendation systems at the end of last year given the app's text-based content, and have already seen a 4% lift in time spent from the first launch.",
        "As we do for any newly monetized surface, we expect to gradually ramp ad supply as we optimize the ad formats and ensure they feel native to the app.",
    ]
}

#### Loading the datasets

**Loading JSON Vector DB:**

In [ ]:
path = f'{file_path}/{vector_file_name}'

with open(path, "r") as file:
    Vector_db = json.load(file)

In [ ]:
# sample of how to access
Vector_db['FOMC_min_JAN25.txt'][0]['chunk_text']

#### Embedding Model

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(EMBEDDING_MODEL)

**Version that handles duplicate chunks:**

In [ ]:
flat_chunks = []
all_embeddings = []
chunk_metadata = []
seen_chunks = {}  # set of chunk texts

for doc_name, chunks in Vector_db.items():
    for chunk in chunks:
      chunk_text = chunk['chunk_text']

      # Check if we've seen exact text before
      if chunk_text not in seen_chunks:
          # First occurrence: add to set
          seen_chunks[chunk_text] = len(flat_chunks)

          # normal processing
          flat_chunks.append(chunk_text)
          all_embeddings.append(chunk['embed'])
          chunk_metadata.append({
              'source_document': doc_name,
              'chunk_id': chunk['chunk_id'],
              'original_index': len(flat_chunks) - 1
          })
      else:
          # Duplicate, comment out to avoid log
          print(f"Skipping duplicate from {doc_name}, chunk_id: {chunk['chunk_id']}")
          # continue  # need this statment if cooment out log

In [ ]:
len(flat_chunks), len(all_embeddings), len(chunk_metadata) # 2000 less chunks than before (for sentence VectorDB)

In [ ]:
# instead of cosine similarity we are taking the dot product
# this code is part of sentence chunking and create json
import faiss
import numpy as np

vecs = np.asarray(all_embeddings, dtype=np.float32)
print(f"Loaded {len(flat_chunks)} chunks with embeddings of shape: {vecs.shape}")

index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)         # index used in retreive function
print(f"Indexed {index.ntotal} chunks of text.")

#### Language Model (LLAMA)

In [ ]:
# API call to the model
from huggingface_hub import InferenceClient

client = InferenceClient(model="meta-llama/Llama-3.1-8B-Instruct")

**Baseline (No RAG)**

In [ ]:
def baseline(input_query, client):
    instruction_prompt = '''You are a helpful chatbot. Answer using your general knowledge.'''

    try:
        messages = [
            {'role': 'system', 'content': instruction_prompt},
            {'role': 'user', 'content': input_query},
        ]

        response = client.chat_completion(messages=messages, max_tokens=512)

        print('Chatbot response:')
        print(response.choices[0].message.content)

    except Exception as e:
        print(f"Error: {e}")

**RAG implementation start:**

In [ ]:
def retrieve(query, top_k):
    query_vector = model.encode([query], normalize_embeddings=True)
    query_vector = query_vector.astype(np.float32)

    scores, indices = index.search(query_vector, top_k)

    results = []
    for i in range(len(indices[0])):
        idx = indices[0][i]
        score = scores[0][i]

        if idx < len(flat_chunks):
            results.append({
                "text": flat_chunks[idx].strip(),
                "score": float(score),
                "source": f"{chunk_metadata[idx]['source_document']}_chunk_{chunk_metadata[idx]['chunk_id']}"
            })

    return results

In [ ]:
# to extract the source used in the chatbot response
import re

def extract_source(text):
    """Extract all text within square brackets using regex"""
    pattern = r'\[([^\]]+)\]'
    matches = re.findall(pattern, text)
    return matches

In [ ]:
def containment_score(retrieved_chunk, target_excerpt):
    chunk_tokens = set(retrieved_chunk.lower().split())
    target_tokens = set(target_excerpt.lower().split())

    intersection = len(chunk_tokens & target_tokens)
    target_size = len(target_tokens)

    # What fraction of the target is contained in the chunk?
    return intersection / target_size if target_size > 0 else 0

def is_match(retrieved_chunk, target_excerpt, threshold=0.6): # 0.6 is somewhat generous
    # Check if target is substantially contained in chunk
    if containment_score(retrieved_chunk, target_excerpt) >= threshold:
        return True

    # Also check if target appears as exact substring
    if target_excerpt.lower() in retrieved_chunk.lower():
        return True

    return False

def calculate_precision_recall_retrieve(target_passages, k_retrieved_chunks):
  '''
  Calculates precison and recall of a single pair of target passages
  and its corresponding k_retrieved passages that come from the retrive()
  function.

  Uses containment_score() and is_match() functions. to determine if the retrieved
  chunk has enough similar words as the target passage

  '''
  match_cnt = 0
  for tgt in target_passages:
    for rtr in k_retrieved_chunks:
      if is_match(rtr, tgt): # above function
        match_cnt+=1
        # print("Match:")
        # print("Retrieved:",rtr)
        # print("Target:", tgt, "\n")
  recall = match_cnt/len(target_passages)
  precision = match_cnt/len(k_retrieved_chunks)
  return [precision, recall]

In [ ]:
def rag_implementation(input_query, client, target_passages = None, top_k = 100):
    retrieved_knowledge = retrieve(input_query, top_k) # here we can manipulate top_k values

    '''print('Retrieved knowledge:')
    for idx, hit in enumerate(retrieved_knowledge, 1):
        print(f' -(similarity: {hit["score"]:.2f}) [{hit["source"]}] {hit["text"]}')'''

    # build the system prompt with source tags
    context_lines = [f' - [{h["source"]}] {h["text"]}' for h in retrieved_knowledge]
    retrieved_docs = [f'{h["source"]}' for h in retrieved_knowledge]
    retrieved_text = [f'{h["text"]}' for h in retrieved_knowledge]

    # immediatly calculate the precision/recall for this query:
    if target_passages is not None:
        precision_recall_pair = calculate_precision_recall_retrieve(target_passages, k_retrieved_chunks = retrieved_text)
        print("Precision:", precision_recall_pair[0])
        print("Recall:", precision_recall_pair[1])
    else:
        precision_recall_pair = None


    instruction_prompt = (
        "You are a helpful chatbot.\n"
        "Use only the following pieces of context to answer the question. "
        "Cite the source ID in square brackets after each fact:\n"
        + "\n".join(context_lines)
    )

    try:
        messages = [
                {'role': 'system', 'content': instruction_prompt},
                {'role': 'user', 'content': input_query},
            ]

        response = client.chat_completion(messages=messages, max_tokens=512)
        response_text = response.choices[0].message.content
        relevant_docs = extract_source(response_text)

        print('\nChatbot response:')
        print(response_text)

    except Exception as e:
        print(f"Error: {e}")

    # response_text = "Dummy Chatbot Response Text, (Avoiding Usage Limit)"
    # relevant_docs = ["DUMMY:    FOMC_min_JAN25.txt_chunk_17"]

    return retrieved_docs, relevant_docs, retrieved_text, precision_recall_pair, response_text

**Manually Query the ChatBot:**

In [ ]:
input_query = input('\nAsk me a question: ')

print('-'*50)
print('BASELINE:')
baseline(input_query, client)

print('-'*50)
print('RAG IMPLEMENTATION:')
_, _, _, _, _ = rag_implementation(input_query, client)

#### Precision and Recall

**Target json file:** Create each time you run an experiment.

In [ ]:
import json
import os

# delete if exists
path_log = f'{file_path}/target_sample.json'
if os.path.exists(path_log):
  os.remove(path_log)

In [ ]:
def create_initial_json(filename="data.json"):
    """Create initial JSON file with empty lists"""
    initial_data = {
        "queries": [],
        "retrieved_docs": [],
        "relevant_docs": [],
        "reference_answers": [],
        "retrieved_answers": [],
        "precision_recall_pair": [],  #new
        "generated_answers": []
    }

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(initial_data, f, indent=2)
    print(f"Created {filename}")

file_name = f'{file_path}/target_sample.json'
create_initial_json(file_name)

In [ ]:
# not changed
def append_to_json(filename, key, value):
    """Append value to specified key in JSON file"""
    # Load existing data
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Append new value
    if key in data and isinstance(data[key], list):
        data[key].append(value)

    # Save updated data
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Added '{value}' to '{key}'")

In [ ]:
TOP_K = 100 # for experiments
for query, answers in queries_and_target.items():
    # now pass 'answers' variable to rag_implementation() to calculate precision_recall for retrival
    retrieved_docs, relevant_docs, retrieved_ans, precision_recall_pair, response_text = rag_implementation(query, client, answers, TOP_K)
    append_to_json(file_name, "queries", query)                          # the actual query
    append_to_json(file_name, "retrieved_docs", retrieved_docs)          # the top-k retrieved chunks IDs as list of string, exp "FOMC_min_JAN25.txt_chunk_57"
    append_to_json(file_name, "relevant_docs", relevant_docs)            # the chunks IDs that were actually used and cited in the response., also like "FOMC_min_JAN25.txt_chunk_57"
    append_to_json(file_name, "reference_answers", answers)              # the actual text target_passages corresponding to the query
    append_to_json(file_name, "retrieved_answers", retrieved_ans)        # the top-k chunks as a list of strings (now the actual chunk text)
    append_to_json(file_name, "precision_recall_pair", precision_recall_pair) # new
    append_to_json(file_name, "generated_answers", response_text)        # chat-bot response

**Precision and Recall**

In [ ]:
def calculate_precision_recall_output(queries, retrieved_docs, relevant_docs):
    # Calculate precision and recall for a list of queries.
    total_precision = 0
    total_recall = 0
    num_queries = len(queries)

    for i in range(num_queries):
        retrieved = set(retrieved_docs[i])
        relevant = set(relevant_docs[i])

        if not retrieved:
            precision = 0
        else:
            precision = len(retrieved.intersection(relevant)) / len(retrieved)

        if not relevant:
            recall = 0
        else:
            recall = len(retrieved.intersection(relevant)) / len(relevant)

        total_precision += precision
        total_recall += recall

    average_precision = total_precision / num_queries
    average_recall = total_recall / num_queries

    return {"average_precision": average_precision, "average_recall": average_recall}

In [ ]:
import json

with open(file_name, 'r') as f:
    data = json.load(f)


queries = data["queries"]
retrieved_docs = data["retrieved_docs"]
relevant_docs = data["relevant_docs"]
retrieved_chunks = data["retrieved_answers"]
reference_answers = data["reference_answers"]
precision_recall_pair = data["precision_recall_pair"]
generated_response = data["generated_answers"]

####**Precision Recall For Retrival:**

In [ ]:
mean_prec_retr = np.mean(np.array(precision_recall_pair)[:, 0])
mean_recall_retr = np.mean(np.array(precision_recall_pair)[:, 1])
print("VectorDB used:", vector_file_name)
print("Top-K value used for experiment:", TOP_K)
print("Mean Precision for Retrival:", round(mean_prec_retr, 4))
print("Mean Recall for Retrival:", round(mean_recall_retr,4))

####**Precision Recall For Output:**

In [ ]:
metrics = calculate_precision_recall_output(queries, retrieved_docs, relevant_docs)
print(metrics) # from old run

#### ROUGE and BLEU score

In [ ]:
from rouge_score import rouge_scorer
import statistics

def calculate_rouge(text1, text2):
    """
    Calculate ROUGE scores between two texts.
    Returns a dictionary with rouge1, rouge2, and rougeL scores.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(text1, text2)

    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu(text1, text2):
    """
    Calculate BLEU score between two texts.
    """
    reference = [text1.split()]
    candidate = text2.split()
    bleu_score = sentence_bleu(reference, candidate)

    return bleu_score

##### **Between generated and reference texts**
This calculates the similarity between the final answer generated by the language model (after considering the retrieved chunks) and your manually created reference answers. This measures the quality of the generated response and how well it captures the information in the reference answers. It tells you how good your system's output is.

In [ ]:
# Download NLTK data if not already downloaded
try:
    nltk.data.find('tokenizers/punkt')
except LookupError: # Changed from nltk.downloader.DownloadError to LookupError
    nltk.download('punkt')

rouge_scores_generated_vs_reference = []
bleu_scores_generated_vs_reference = []

for i in range(len(generated_response)):
    generated = generated_response[i]
    references = reference_answers[i]

    rouge_scores_per_query = []
    bleu_scores_per_query = []

    for reference in references:
        # Calculate ROUGE scores
        rouge_score = calculate_rouge(reference, generated)
        rouge_scores_per_query.append(rouge_score)

        # Calculate BLEU scores
        bleu_score = calculate_bleu(reference, generated)
        bleu_scores_per_query.append(bleu_score)

    # Average ROUGE scores for the current query
    avg_rouge_rouge1 = statistics.mean([score['rouge1'] for score in rouge_scores_per_query])
    avg_rouge_rouge2 = statistics.mean([score['rouge2'] for score in rouge_scores_per_query])
    avg_rouge_rougel = statistics.mean([score['rougeL'] for score in rouge_scores_per_query])

    rouge_scores_generated_vs_reference.append({
        'rouge1': avg_rouge_rouge1,
        'rouge2': avg_rouge_rouge2,
        'rougeL': avg_rouge_rougel
    })

    # Average BLEU scores for the current query
    avg_bleu_score = statistics.mean(bleu_scores_per_query)
    bleu_scores_generated_vs_reference.append(avg_bleu_score)

# Calculate overall average ROUGE and BLEU scores across all queries
overall_avg_rouge1 = statistics.mean([score['rouge1'] for score in rouge_scores_generated_vs_reference])
overall_avg_rouge2 = statistics.mean([score['rouge2'] for score in rouge_scores_generated_vs_reference])
overall_avg_rougel = statistics.mean([score['rougeL'] for score in rouge_scores_generated_vs_reference])
overall_avg_bleu = statistics.mean(bleu_scores_generated_vs_reference)

print("Average ROUGE scores (Generated vs Reference):")
print(f"  ROUGE-1: {overall_avg_rouge1:.4f}")
print(f"  ROUGE-2: {overall_avg_rouge2:.4f}")
print(f"  ROUGE-L: {overall_avg_rougel:.4f}")
print("\nAverage BLEU score (Generated vs Reference):")
print(f"  BLEU: {overall_avg_bleu:.4f}")

##### **Between retrieved chunks and reference texts**
This calculates the similarity between the individual chunks of text that were retrieved from your knowledge base and your reference answers. This measures the quality of the retrieval process and how well the system is finding relevant information. It tells you if the RAG system is providing the language model with useful context.

In [ ]:
rouge_scores_retrieved_vs_reference = []
bleu_scores_retrieved_vs_reference = []

for i in range(len(retrieved_chunks)):
    retrieved_chunk_list = retrieved_chunks[i]
    references = reference_answers[i]

    rouge_scores_per_query = []
    bleu_scores_per_query = []

    for retrieved_chunk in retrieved_chunk_list: # Iterate through individual retrieved chunks
        for reference in references:
            # Calculate ROUGE scores
            rouge_score = calculate_rouge(reference, retrieved_chunk)
            rouge_scores_per_query.append(rouge_score)

            # Calculate BLEU scores
            bleu_score = calculate_bleu(reference, retrieved_chunk)
            bleu_scores_per_query.append(bleu_score)

    # Average ROUGE scores for the current query
    avg_rouge_rouge1 = statistics.mean([score['rouge1'] for score in rouge_scores_per_query])
    avg_rouge_rouge2 = statistics.mean([score['rouge2'] for score in rouge_scores_per_query])
    avg_rouge_rougel = statistics.mean([score['rougeL'] for score in rouge_scores_per_query])

    rouge_scores_retrieved_vs_reference.append({
        'rouge1': avg_rouge_rouge1,
        'rouge2': avg_rouge_rouge2,
        'rougeL': avg_rouge_rougel
    })

    # Average BLEU scores for the current query
    avg_bleu_score = statistics.mean(bleu_scores_per_query)
    bleu_scores_retrieved_vs_reference.append(avg_bleu_score)


# Calculate overall average ROUGE and BLEU scores across all queries
overall_avg_rouge1 = statistics.mean([score['rouge1'] for score in rouge_scores_retrieved_vs_reference])
overall_avg_rouge2 = statistics.mean([score['rouge2'] for score in rouge_scores_retrieved_vs_reference])
overall_avg_rougel = statistics.mean([score['rougeL'] for score in rouge_scores_retrieved_vs_reference])
overall_avg_bleu = statistics.mean(bleu_scores_retrieved_vs_reference)

print("Average ROUGE scores (Retrieved vs Reference):")
print(f"  ROUGE-1: {overall_avg_rouge1:.4f}")
print(f"  ROUGE-2: {overall_avg_rouge2:.4f}")
print(f"  ROUGE-L: {overall_avg_rougel:.4f}")
print("\nAverage BLEU score (Retrieved vs Reference):")
print(f"  BLEU: {overall_avg_bleu:.4f}")